In [6]:
import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False
# stanza.download('ru')
import pandas as pd
from tqdm import tqdm
import stanza
import json
import pymorphy2
from nltk.tokenize import sent_tokenize

In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
def load_stop_words():
    stopwords = []
    path_to_file = "stopwords/Stopwords.txt"
    with open(path_to_file, "r", encoding="utf-8") as fl:
        for line in fl:
            stopwords.append(line.strip("\n"))
    return stopwords

In [9]:
def norm_form(morph, word):
    return morph.parse(word)[0].normal_form

## Load preprocessors

In [10]:
morph = pymorphy2.MorphAnalyzer(lang="ru")

In [13]:
nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse', use_gpu = True)

2021-05-28 22:32:35 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-05-28 22:32:35 INFO: Use device: cpu
2021-05-28 22:32:35 INFO: Loading: tokenize
2021-05-28 22:32:35 INFO: Loading: pos
2021-05-28 22:32:35 INFO: Loading: lemma
2021-05-28 22:32:35 INFO: Loading: depparse
2021-05-28 22:32:36 INFO: Loading: ner
2021-05-28 22:32:37 INFO: Done loading processors!


## Load data

In [14]:
stopwords = load_stop_words()

## Data was loaded from source:   https://gorod.mos.ru/

In [15]:
# df = pd.read_csv('temp.csv', sep="$")
df = pd.read_excel('Данные по голосованиям.xlsx')

### Clearing & Filtering data

In [17]:
df["lens"] = df["note"].apply(lambda x: len(str(x)))

In [19]:
df_filtered = df[df["lens"] > 50]

In [20]:
df_filtered.shape

(6437, 12)

In [ ]:
df_filtered["theme_value"].value_counts()

In [ ]:
df_filtered.head()

In [ ]:
df_0 = df_filtered[df_filtered["theme_value"]==0]
df_1 = df_filtered[df_filtered["theme_value"]==1]
df_2 = df_filtered[df_filtered["theme_value"]==2]

## Tokenizing

In [22]:
full_corpus = df_filtered["note"].values

In [30]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


False

In [27]:
try:
    sentences = [sent for corp in full_corpus for sent in sent_tokenize(corp, language="russian")]
except:
    import nltk
    nltk.download('punkt')
    sentences = [sent for corp in full_corpus for sent in sent_tokenize(corp, language="russian")]

[nltk_data] Error loading punkt: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/russian.pickle[0m

  Searched in:
    - 'C:\\Users\\skrochinskiy_ki/nltk_data'
    - 'C:\\Users\\skrochinskiy_ki\\Anaconda3\\nltk_data'
    - 'C:\\Users\\skrochinskiy_ki\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\skrochinskiy_ki\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\skrochinskiy_ki\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
long_sents = [i for i in sentences if len(i) > 20]

In [ ]:
len(long_sents), len(sentences)

## Get Triplets

In [ ]:
%%time
triplets = []
for s in tqdm(long_sents):
    doc = nlp(s)
    for sent in doc.sentences:
            entities = [ent.text for ent in sent.ents]
            res_d = dict()
            temp_d = dict()
            for word in sent.words:
                temp_d[word.text] = {"head": sent.words[word.head-1].text, "dep": word.deprel, "id": word.id}
            for k in temp_d.keys():
                nmod_1 = ""
                nmod_2 = ""
                if (temp_d[k]["dep"] in ["nsubj", "nsubj:pass"]) & (k in entities):
                    res_d[k] = {"head": temp_d[k]["head"]}
                    
                    for k_0 in temp_d.keys():
                        if (temp_d[k_0]["dep"] in ["obj", "obl"]) &\
                           (temp_d[k_0]["head"] == res_d[k]["head"]) &\
                            (temp_d[k_0]["id"] > temp_d[res_d[k]["head"]]["id"]):
                            res_d[k]["obj"] = k_0
                            break
                    
                    for k_1 in temp_d.keys():
                        if (temp_d[k_1]["head"] == res_d[k]["head"]) & (k_1 == "не"):
                            res_d[k]["head"] = "не "+res_d[k]["head"]
                    
                    if "obj" in res_d[k].keys():
                        for k_4 in temp_d.keys():
                            if (temp_d[k_4]["dep"] =="nmod") &\
                               (temp_d[k_4]["head"] == res_d[k]["obj"]):
                                nmod_1 = k_4
                                break
                                
                        for k_5 in temp_d.keys():
                            if (temp_d[k_5]["dep"] =="nummod") &\
                               (temp_d[k_5]["head"] == nmod_1):
                                nmod_2 = k_5
                                break
                        res_d[k]["obj"] = res_d[k]["obj"]+" "+nmod_2+" "+nmod_1

            if len(res_d) > 0:
                triplets.append([s, res_d])

In [ ]:
clear_text = lambda x: "".join(i if (i.isdigit()) | (i.isalpha()) | (i in [" "]) else " " for i in x )

clear_triplets = dict()
for tr in triplets:
    for k in tr[1].keys():
        if "obj" in tr[1][k].keys():
            ## clear_text убрать, если не нужна очистка предложений
            clear_triplets[clear_text(tr[0])] =  [k, tr[1][k]['head'], tr[1][k]['obj']] 

In [ ]:
for_df = []
for k in clear_triplets.keys():
    for_df.append([k]+clear_triplets[k])

## Create DF for prepare

In [ ]:
df_triplets = pd.DataFrame(for_df, columns=["full_sent", "subject", "verb", "object"])
df_triplets.shape

In [ ]:
df_triplets["subj_n_f"] = df_triplets["subject"].apply(lambda x: norm_form(morph, x))
df_triplets["obj_n_f"] = df_triplets["object"].apply(lambda x: norm_form(morph, x))

In [ ]:
df_triplets.head(5)

In [ ]:
df_filtered = df_triplets[(~df_triplets["subj_n_f"].isin(stopwords)) &\
                          (~df_triplets["obj_n_f"].isin(stopwords))].sort_values(by="obj_n_f", ascending=False, ignore_index=True)

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.head(3)

## Split data on chunks

In [ ]:
groups = list(chunks(df_filtered["obj_n_f"].unique(), 100))
len(groups)

In [ ]:
gr_num = 0
df_for_draw = df_filtered[df_filtered["obj_n_f"].isin(groups[gr_num])]

In [ ]:
nodes = pd.unique(df_for_draw[["subj_n_f", "obj_n_f"]].values.ravel("K"))

In [ ]:
nodes.shape

## Get edges & edges info

In [ ]:
df_d_d = df_for_draw.drop_duplicates(subset=["subj_n_f", "obj_n_f", "verb"])[["subj_n_f", "obj_n_f", "verb", "full_sent"]]

In [ ]:
df_d_d.shape, df_for_draw.shape

In [ ]:
info_dict = dict()
label_dict = dict()
for cc, raw in enumerate(df_d_d.values):
    info_dict[(raw[0], raw[1])] = {f"sent_{cc}": raw[3]}
    label_dict[(raw[0], raw[1])] = raw[2]

In [ ]:
word_num = dict()
for c, word in enumerate(nodes):
    word_num[word] = c+1

## Draw Graph

In [ ]:
import script_for_graph
import importlib
importlib.reload(script_for_graph)
from script_for_graph import header_text, tail_text

header_text += """\nvar nodes = new vis.DataSet([\n"""
for w in nodes:
    header_text += "{"
    header_text += f"""         id: {word_num[w]}, 
                                label: "{w}"\n"""
    header_text += "},"
header_text += "   ]);\n"

header_text += """var edges = new vis.DataSet(["""
for k in info_dict.keys():
    header_text += "{"
    header_text += f"""       from: {word_num[k[0]]}, 
                    to: {word_num[k[1]]}, 
                    arrows: "to",
                    label: "{label_dict[k]}",
                    info: {info_dict[k]}\n"""
    header_text +="},"
header_text += "   ]);\n"

full_text = ""
full_text += header_text
full_text += tail_text

with open(f"Graph_for_group_{gr_num}.html", "w", encoding="utf-8") as f: 
    f.write(full_text)